# Collective Synchronization: The Kuramoto Model

Author: Ravi Chepuri

This notebook is the second part of a challenge question for the 2023 Winter Workshop of the [GRAD-MAP program](https://www.umdgradmap.org). The challenge question is intended to be a week-long exploration in nonlinear dynamics for physics students at the intermediate undergraduate level who have limited previous Python and coding experience.

This challenge question is inspired by L Q English 2008 Eur. J. Phys. 29 143.

## III. The order parameter and a phase transition

In part III., we will begin by addressing a rather general question: under what conditions does synchronization occur in the Kuramoto model? We will find that in order to address this question, we need some quantitative measurement of synchronization, which we will call the order parameter. By examining the behavior of the order parameter at different coupling strengths, we can begin to understand the conditions under which synchronization occurs in the Kuramoto model. But perhaps more significantly, we will encounter a phase transition in the order parameter, which is a very deep concept that connects to may other areas of physics.

Note that each task in part III. will be more open ended and possibly more difficult than tasks in part II. 

Before we begin, we must import the same code as we did in the previous notebook:

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from kuramoto_plot import animate_kuramoto

At the end of the last notebook you were optionally tasked to define a function that streamlines the process of running a Kuramoto model simulation:

>**Optional task:** Define a function that takes `N`, `K`, `mean_omega`, `standard_deviation_omega`, `dt`, and `number_of_time_steps` as arguments, and performs a simulation of the Kuramoto model with those parameters, as you've done previously. The function should return a 2D Numpy array `thetas` containing the simulation results, and also return a 1D Numpy array `omegas` containing the natural frequencies of the oscillators.

The solution to this task is provided here, as it will likely be useful in the rest of this notebook.

In [13]:
def simulate_kuramoto(N, K, mean_omega, standard_deviation_omega, dt, number_of_time_steps):
    """Performs a simulation of the Kuramoto model using Euler's method.

    Args:
        N: Number of Kuramoto oscillators
        K: Coupling constant
        mean_omega: Average value of the normal distribution that the natural
            frequencies are drawn from
        standard_deviation_omega: Width of the normal distribution that the
            natural frequencies are drawn from
        dt: Timestep used for Euler's method in the simulation
        number_of_time_steps: Length of simulation in units of dt

    Returns:
        A tuple of two objects. The first is a 2D Numpy array containing the
        results of the simulation (element [i, n] is the phase of oscillator i+1
        at time t_n). The second is a 1D Numpy array containing a list of the
        natural frequencies of the N oscillators.
    """
    omegas = np.random.default_rng().normal(mean_omega, standard_deviation_omega, (N))
    thetas = np.zeros((N, number_of_time_steps))
    thetas[:, 0] = 2 * np.pi * np.random.rand(N)
    for n in range(1, number_of_time_steps):
        for i in range(N):
            thetas[i, n] = thetas[i, n-1] + (omegas[i] + K/N * sum(np.sin(thetas[j, n-1] - thetas[i, n-1]) for j in range(N))) * dt
    return thetas, omegas

### Instances where synchronization does not occur in the Kuramoto model

If you haven't already, let's adjust some of the parameters for our $N=32$ Kuramoto model simulation, and see if we can observe a few instances in which synchronization does and doesn't occur.

**Task:** Use the `simulate_kuramoto` function to simulate the case when there are $N=32$ Kuramoto oscillators and the $\omega_i$ are picked from a normal distribution with mean $1$ and standard deviation $0.1$, in 3 cases:
1. When $K = 0.05 \mathrm{Hz}$
2. When $K = 0.17 \mathrm{Hz}$
3. When $K = 0.5 \mathrm{Hz}$

Use the same simulation parameters as in the previous notebook (timestep $\Delta t = 0.1 \mathrm{s}$ and a simulation length of $1000$ time steps). Create an animation for each case. What do you observe?

(Note: `simulate_kuramoto` returns two objects: `thetas`, which is a 2D Numpy array holding the results of the simulation, and `omegas`, which is a 1D Numpy array holding the natural frequencies of the oscillators used in the simulation. To get these objects, use code of the form `thetas, omegas = simulate_kuramoto(...)`, and to animate the results use code of the form `animate_kuramoto(*thetas, natural_frequencies=omegas)`.)

In [4]:
### YOUR CODE HERE ###
# You may insert additional cells if needed

**Reflection:** Describe the collective motion of the thirty-two oscillators in each of the three cases.

*Insert your reflection here*

If you create your animations correctly, you should observe that there can be different degrees of synchronization in the Kuramoto model. To investigate the conditions under which synchronization occurs, it would be nice to have a way to quantify the amount of synchronization that we observe. This motivates the introduction of an "order parameter," described below.

### Average position of oscillators

One way to understand the synchronization of the Kuramoto oscillators is to look at the average positions of the oscillators on the unit circle.

**Task:** Modify your code from the three cases in the previous section by additionally passing the argument `show_average=True` into the `plot_kuramoto` function. This should display the average position of the Kuramoto oscillators. What do you observe in each of the three cases?

In [ ]:
### YOUR CODE HERE ###
# You may insert additional cells if needed

**Reflection:** Describe the motion of average position of the oscillators in each of the three cases. What can the average position tell you about the collective behavior of the 32 individual oscillators?

*Insert your reflection here*

### The order parameter

In the animations in the previous section, you should see that the average position of the oscillators stays relatively close to the center of the unit circle (the origin) when the oscillators are not synchronized. On the other hand, the average position goes close to the edge of the unit circle when there is some sychronization! As such, we consider the distance from the average position of the oscillators to the origin as a measure of the amount of synchronization there is. We call this distance the order parameter $r$:
\begin{align*}
    r &= \text{distance from avg. position of oscillators to origin} \\
      &= \sqrt{x_{avg}^2 + y_{avg}^2}.
\end{align*}
where the second equality comes from the Pythagorean theorem. Using unit circle trigonometry, we know that the $x$ and $y$ positions of the $i$ th oscillator are $x_i = \cos(\theta_i)$ and $y_i = \sin(\theta_i)$, so by taking an average, we can find that $x_{avg} = \frac{1}{N} \sum_{i=1}^N \cos(\theta_i)$ and $y_{avg} = \frac{1}{N} \sum_{i=1}^N \sin(\theta_i)$. Plugging into the above equation and simplifying gives an expression for the order parameter in terms of the $\theta_i$ s:

\begin{equation}
  r = \frac{1}{N} \sqrt{\left(\sum_{i=1}^N \cos(\theta_i)\right)^2 + \left(\sum_{i=1}^N \sin(\theta_i)\right)^2}
\end{equation}

Given that we know the values of all the $\theta_i$ s at some time $t$, we can calculate $r$ at that time $t$. If $r$ is close to $0$, that means that the average position of the oscillators is near the origin and there is not much synchronization at that time. On the other hand, if $r$ is close to $1$, that means that the average position of the oscillators is near the edge of the unit circle and there is synchronization at that time.

Note: In some literature, you may see the order parameter defined using complex numbers: $r = \frac{1}{N} \left| \sum_{i=1}^{N} e^{i \theta_i} \right|$. This definition is equivalent, and slightly more straightforward to calculate, if you have familiarity working with complex numbers.

**Task:** Define a function that calculates the order parameter of the results of a Kuramoto simulation at every timestep. It should take a 2D Numpy array `thetas` as its argument, and produce a 1D array `order_params` that contains values of $r(t)$ for $t = 0 * \Delta t,\, t = 1 * \Delta t,\, \ldots,\, t = (\text{number of time steps}) * \Delta t$. *Hint:* Use Numpy vectorization to your advantage.

**Optional task:** Rewrite the same function using complex numbers.

In [28]:
def order_parameter(thetas):
    """Calculates the order parameter of the results of a Kuramoto simulation at
    every time step.

    Args:
        thetas: a 2D Numpy array containing the results of the simulation
            (element [i, n] is the phase of oscillator i+1 at time t_n)

    Returns:
        A 1D Numpy array containing the order parameter at every timestep
    """
    ### YOUR CODE BELOW ###
    order_params = None
    return order_params

### Time evolution of the order parameter

Let's now look at how the order parameter $r$ changes over the course of our simulations.

**Task:** For cases 1, 2, and 3 that you simulated previously, create a plot of the order parameter over time. What do you observe in each of the three cases? *Hint:* Use the package `matplotlib` to produce your plot. Look up numerous online resources for plotting functions using `matplotlib`. 

In [10]:
### YOUR CODE HERE ###

**Reflection:** Describe how the order parameter changes over the course of the simulation for each of the three cases. Does this make sense given what you observed in the animations? Comment on any differences in the behavior near the end of the simulation.

*Insert your reflection here*

### A more numerically efficient Kuramoto model simulation

We introduced the order parameter $r$ as a way to measure the amount of synchronization in a set of Kuramoto oscillators. However, the order parameter turns out to be even more useful: it can be shown that the Kuramoto equations can be rewritten in a form that depends on the order parameter, as we will show below. This is useful because the resulting equations turn out to be quicker to solve numerically than the original Kuramoto equations, allowing us to do simulations with many more Kuramoto oscillators (larger $N$).

Recall that we defined $r$ as the distance from the average position of the oscillators to the origin. Let's also define a quantity $\psi$ as the phase of the average position of the oscillators, if that average position was itself considered as an oscillator.

If you have familiarity with complex numbers, an equivalent definition of $r$ and $\psi$ is given by setting $r e^{i \psi} = \frac{1}{N} \sum_{i=1}^{N} e^{i \theta_i}$. By performing a series of mathematical operations, the Kuramoto equations can be re-expressed in the form
\begin{equation}
    \frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + K r \sin(\psi - \theta_i) \quad (i = 1, \ldots, N).
\end{equation}
(A mathematical proof of this claim is given in an appendix at the end of this notebook.)

This form of the Kuramoto equations is convenient for our computational simulations because we do not have to calculate a sum in order to find each of the $\frac{\mathrm{d} \theta_i}{\mathrm{d} t}$, allowing us to scale to much larger $N$.

**Task**: Rewrite the function `simulate_kuramoto`, but using the order-parameter-based Kuramoto equations provided above. Call this new function `simulate_kuramoto_op`. As a reminder, this was the specification for `simulate_kuramoto`:

>Define a function that takes `N`, `K`, `mean_omega`, `standard_deviation_omega`, `dt`, and `number_of_time_steps` as arguments, and performs a simulation of the Kuramoto model with those parameters. The function should return a 2D Numpy array `thetas` containing the simulation results, and also return a 1D Numpy array `omegas` containing the natural frequencies of the oscillators.

A helper function is given that calculates $\psi$ at each time step. You will also need to use your `order_parameter` function from before. After you're done, create an animation of the results of `simulate_kuramoto_op` and check that it works as expected.

In [16]:
def psi(thetas):
    """Calculates the average phase \psi of the results of a Kuramoto
    simulation at every time step.

    Args:
        thetas: a 2D Numpy array containing the results of the simulation
            (element [i, n] is the phase of oscillator i+1 at time t_n)

    Returns:
        A 1D Numpy array containing the variable \psi at every timestep
    """
    N = thetas.shape[0]
    z = 1/N * np.sum(np.exp(1j * thetas), axis=0)
    return np.angle(z)


def simulate_kuramoto_op(N, K, mean_omega, standard_deviation_omega, dt, number_of_time_steps):
    """Performs a simulation of the Kuramoto model using Euler's method.
    Uses the order parameter governed equations, rather than the original
    Kuramoto equations.

    Args:
        N: Number of Kuramoto oscillators
        K: Coupling constant
        mean_omega: Average value of the normal distribution that the natural
            frequencies are drawn from
        standard_deviation_omega: Width of the normal distribution that the
            natural frequencies are drawn from
        dt: Timestep used for Euler's method in the simulation
        number_of_time_steps: Length of simulation in units of dt

    Returns:
        A tuple of two objects. The first is a 2D Numpy array containing the
        results of the simulation (element [i, n] is the phase of oscillator i+1
        at time t_n). The second is a 1D Numpy array containing a list of the
        natural frequencies of the N oscillators.
    """
    ### YOUR CODE BELOW ###
    thetas = None
    omegas = None
    return thetas, omegas

**Reflection:** Try simulating more and more Kuramoto oscillators by increasing $N$. Compare how long it takes `simulate_kuramoto` (which uses the original Kuramoto equations) and `simulate_kuramoto_op` (which uses the modified Kuramoto equations based on the order parameter) to run the simulations (not including the time it takes to create an animation `animate_kuramoto`). What do you observe?

*Insert your reflection here*

This more efficient simulation method allows us to analyze systems of much more Kuramoto oscillators.

**Task:** Use your new `simulate_kuramoto_op` function to reproduce the plots of the order parameter over time for each of the three cases, but this time with $N=512$ Kuramoto oscillators. (Do NOT animate the results of the simulation, as the animation process will likely take your computer a very long time to complete.)

In [36]:
### YOUR CODE HERE ###

### Getting the "final" value of the order parameter

When we study the Kuramoto model (or many systems in physics), we often care about the long run behavior of the system, rather than the early behavior of the system. This is because the early behavior is determined by the initial conditions, which we can set to be whatever we want. In contrast, in the long run the initial conditions don't matter, so the long run reveals the "true physics" of the system.

More concretely, we saw that in cases where the Kuramoto oscillators experience collective synchronization, it takes some finite amount of time to acheive this sychronization. This can be seen in the plot of the order parameter $r$, which starts out much below $1$ but after some time eventually saturates to a constant number just below $1$. In contrast, when there is not much synchronization, $r$ does not approach a constant number but instead fluctuates around numbers less than $1$. As we can see, the long-run behaviors of $r$, rather than the initial behaviors, tell us about the different phenomena (synchronization vs. no synchronization) observed.

**Task:** Write a function that takes in the results of a Kuramoto model simulation (produced by `simulate_kuramoto` or `simulate_kuramoto_op`), and returns the average order parameter $r$ over the last 20% of the simulation.

In [ ]:
def avg_final_op(thetas, fraction=0.2):
    """Gets the average value of the order parameter over the last 20% (or
    other fraction) of the Kuramoto model simulation.

    Args:
        thetas: a 2D Numpy array containing the results of the simulation
            (element [i, n] is the phase of oscillator i+1 at time t_n)
        fraction: the fraction of the simulation for which to calculate the
            average order parameter over (default: 0.2 i.e. 20%)

    Returns:
        A single number that is the average value of the order parameter
        over the last 20% (or other fraction) of the Kuramoto model simulation.
    """
    ### YOUR CODE BELOW ###
    return 0

**Task:** Use the function `avg_final_op` to get the average value of the order parameter over the last $20%$ of the simulations of each of the three cases you considered previously (with $N=512$). Check that these values approximately match the plot you produced in the previous section about the time evolution of the order parameter.

In [33]:
### YOUR CODE HERE ###

### Bifurcation diagram

So far, we've only considered three cases for the value of the coupling constant $K$. Let's now try to understand the long-run behavior of $r$ at a wide range of $K$ values. We'll first pick many different values of $K$, then use our previous code to run a simulation of the Kuramoto model with that $K$ and get the long run value of $r$, and finally display the results.

For our values of $K$, it's often a good choice to choose a range of values that we want to consider, and then pick evenly spaced values throughout this range. In this context, a decent range of values to try is $K$ between $0.00$ and $1.00$. If we pick a spacing of $0.01$, then we will have $101$ different values of $K$ to consider: $K = 0.00, 0.01, 0.02, \ldots, 0.99, 1.00$. 

Next, we loop over all the values of $K$. For each value of $K$, we run a Kuramoto model simulation, and then calculate the long run value of the order parameter.

Finally, how should we display our data? A illustrative way of doing this is to create a plot with values of $K$ on the horizontal axis, and values of long-run $r$ on the vertical axis. Such a plot is known as a bifurcation diagram.

**Task:** Create a bifurcation diagram for $N=512$ Kuramoto oscillators with $K$ ranging from $0.00$ to $1.00$ with a spacing of $0.01$. As you've done previously, the natural frequencies should be drawn from a normal distribution with mean $1$ and standard deviation $0.1$. You should use the same timestep for the simulation as you have previously, but use a simulation length of $5000$ timesteps now (so the oscillators have time to settle into their long-run behavior). The long-run order parameter should be estimated by taking the average of order parameter over the last $20%$ of the simulation.

(*Hint:* Use Numpy's `linspace` function to create an array of evenly spaced values of $K$ for you to loop over.)

(*Hint 2:* Use Matplotlib's `plt.scatter` function to plot data points.)

(*Hint 3:* Since you are doing $101$ separate simulations, it may take your computer a while to produce a complete bifurcation diagram, depending on your computer, but it shouldn't take more than half and hour.)

In [ ]:
### YOUR CODE HERE ###

**Reflection:** What do you observe about the shape of the bifurcation diagram? Does the plot appear continuous or discontinuous? Does the long-run $r$ have an increasing or decreasing relationship with $K$, or neither? If there is a relationship, does it appear linear or non-linear? Are there ranges of values of $K$ where adjusting $K$ has limited effect on the long-run $r$? How about ranges of values of $K$ where adjusting $K$ has a large impact on the long-run $r$?

*Insert your reflection here*

You may have noticed that there is a relatively sharp cutoff between values of $K$ for which the long-run $r$ is near $0$ and values of $K$ for which  the long-run $r$ is near $1$. This phenomenon is actually an example of a phase transition, an important concept that appears in many areas of physics.

A phase transition is a sudden change in the behavior or properties of a system that occurs at a particular critical point or threshold. It is usually accompanied by a change in the system's symmetry or order. For instance, when liquid water is cooled past 0 degrees Celcius, it suddenly becomes more ordered, forming a crystal structure and becoming ice.

The key feature of the Kuramoto model that you have observed in the bifurcation diagram is that it exhibits a phase transition from an incoherent state, where the oscillators are not synchronized, to a ordered state, where the oscillators are synchronized. This phase transition occurs at a critical coupling strength of about $K \approx 0.16$.

At low coupling strength, the oscillators are only weakly influenced by each other and do not synchronize. As the coupling strength increases, the oscillators become more strongly influenced by each other and begin to synchronize. At a critical coupling strength, the oscillators suddenly synchronize and behave as a single, coherent entity. This sudden change in behavior is a phase transition.

### Key observations

Use this section to record your main takeaways from this notebook.

*Insert your reflection here*

## Appendix: Deriving the order parameter governed Kuramoto equations

In this appendix, we derive the order parameter governed Kuramoto equations using complex analysis. You can complete this notebook without understanding this derivation, but it is provided here for completeness.


***Claim:*** With $r$ and $\psi$ defined by 
\begin{equation}
    r e^{i \psi} = \frac{1}{N} \sum_{j=1}^{N} e^{i \theta_j},
\end{equation}
the Kuramoto equations
\begin{equation}
    \frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} \sin(\theta_j - \theta_i) \quad (i = 1, \ldots, N)
\end{equation}
can equivalently be expressed as
\begin{equation}
    \frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + K r \sin(\psi - \theta_i) \quad (i = 1, \ldots, N).
\end{equation}

***Proof:*** Starting with $r e^{i \psi} = \frac{1}{N} \sum_{j=1}^{N} e^{i \theta_j}$, multiply both sides by $e^{-i \theta_i}$ (for any $i=1,2,\ldots,N$) to obtain
\begin{equation}
    r e^{i(\psi - \theta_i)} = \frac{1}{N} \sum_{j=1}^N e^{i(\theta_j - \theta_i)}.
\end{equation}
Taking the imaginary part of both sides gives
\begin{equation}
    r \sin{(\psi - \theta_i)} = \frac{1}{N} \sum_{j=1}^N \sin{(\theta_j - \theta_i)}.
\end{equation}
The $i$ th Kuramoto equation can be rearranged to state
\begin{equation}
    \frac{1}{N} \sum_{j=1}^N \sin{(\theta_j - \theta_i)} = \frac{\frac{\mathrm{d} \theta_i}{\mathrm{d} t} - \omega_i}{K},
\end{equation}
so plugging into the above equation gives
\begin{equation}
    r \sin{(\psi - \theta_i)} = \frac{\frac{\mathrm{d} \theta_i}{\mathrm{d} t} - \omega_i}{K}
\end{equation}
which can be rearranged to yield
\begin{equation}
    \frac{\mathrm{d} \theta_i}{\mathrm{d} t} = \omega_i + K r \sin(\psi - \theta_i). \quad \blacksquare
\end{equation}